###Task 2: Building a custom Web Scarpper for searching youtube channel links###

####***Author:*** Shirshakk Purkayastha####


#####Script 1: Gathering via HTMLSession() method#####

Installing the required Dependencies

In [ ]:
!pip install requests-html
!pip install bs4

Importing Libraries

In [ ]:
import requests
import urllib
import csv
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession, AsyncHTMLSession
from bs4 import BeautifulSoup as bs
import nest_asyncio

Function to scrape for the google query

In [ ]:
def get_source(url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

def scrape_google(query, num_results=10):
    query = urllib.parse.quote_plus(query)
    response = get_source(f"https://www.google.co.uk/search?q={query}&num={num_results}")

    links = list(response.html.absolute_links)
    google_domains = ('https://www.google.',
                      'https://google.',
                      'https://webcache.googleusercontent.',
                      'http://webcache.googleusercontent.',
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.',
                      'https://youtube.com')

    for url in links[:]:
        if url.startswith(google_domains):
            links.remove(url)

    return links

Gather the links provided by the search query, using the above function

In [ ]:
youtube_links = scrape_google("site:youtube.com openinapp.co", num_results=10000)

Function to gather 'channel names' from the above gathered 'youtube_links'

In [ ]:
channel_names = []
for i in youtube_links:
  try:
    r = requests.get(i)
    page = r.text
    soup = bs(page, 'html.parser')
    vids = soup.find("span", itemprop='author').next.next['content']
    channel_names.append(vids)
  except KeyError:
    channel_names.append("N/A")
  except AttributeError:
    channel_names.append("N/A")

Function to write the procured Youtube Links into '.csv' file

In [ ]:
csv_filename = 'youtube_links.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['YouTube Links'])
    writer.writerows([[name] for name in youtube_links])

Function to write the procured Youtube Channel Links into '.csv' file

In [ ]:
csv_filename = 'youtube_channels.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['YouTube Channel Links'])
    writer.writerows([['https://youtube.com/{}'.format(name)] for name in channel_names])